In [2]:
import os
try:
    %run setup_paths
except:
    %run notebooks/setup_paths
    
print("current dir: ", os.getcwd())

current dir:  c:\Projects\scmsim


In [11]:
%run src/scm

logging.basicConfig(
    level=logging.INFO,  # or DEBUG, WARNING, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)




In [12]:
def show(**kwargs):
    return pd.DataFrame({k:v.unsqueeze(0) for k,v in kwargs.items()})

## Sanity - No causality
(Two items of the same gerne)

In [13]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.2 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.2 + 0.5*U))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

# GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))



2025-07-26 16:31:38,571 - INFO - U : f(['N'])
2025-07-26 16:31:38,573 - INFO - T : f(['N', 'U'])
2025-07-26 16:31:38,574 - INFO - Y : f(['N', 'U', 'T'])
2025-07-26 16:31:38,576 - INFO - fitting model for propensity: 'propensity'
2025-07-26 16:31:38,662 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.249333,-0.000694,-0.000634,-0.000691


## Causality with Confounder

In [14]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.1 + 0.5*U + 0.2*T))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

# GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))


2025-07-26 16:31:39,442 - INFO - U : f(['N'])
2025-07-26 16:31:39,442 - INFO - T : f(['N', 'U'])
2025-07-26 16:31:39,442 - INFO - Y : f(['N', 'U', 'T'])
2025-07-26 16:31:39,442 - INFO - fitting model for propensity: 'propensity'
2025-07-26 16:31:39,515 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.471822,0.198087,0.198185,0.198088


## Causality with a weak Confounder
(For example Godfather-I, Godfather II)

In [15]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.05 + 0.05*U  + 0.5*T))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))


show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))


2025-07-26 16:31:40,286 - INFO - U : f(['N'])
2025-07-26 16:31:40,287 - INFO - T : f(['N', 'U'])
2025-07-26 16:31:40,289 - INFO - Y : f(['N', 'U', 'T'])
2025-07-26 16:31:40,290 - INFO - fitting model for propensity: 'propensity'
2025-07-26 16:31:40,354 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.523457,0.495429,0.495439,0.495422


## Information Leakage

In [16]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.4*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.1 + 0.4*U  + 0.5*T))*1.0,
    "Leakage" : lambda N, T, Y: (T+Y) * (torch.rand(N) < 0.3) 
}

values = eval_scm(SCM, 10000)
enrich_propensity(values, "T", ["U"], "propensity", model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))
enrich_propensity(values, "T", ["U","Leakage"], "propensity_leakage", model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

mgen = lambda: GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"], mgen=mgen)(values),
     ATE_Propensity_Leakage=ATEPropensity("T","Y", "propensity_leakage")(values),
     ATE_Impute_Leakage=ATEImpute("T","Y",["U","Leakage"], mgen=mgen)(values))
     


2025-07-26 16:31:41,101 - INFO - U : f(['N'])
2025-07-26 16:31:41,103 - INFO - T : f(['N', 'U'])
2025-07-26 16:31:41,103 - INFO - Y : f(['N', 'U', 'T'])
2025-07-26 16:31:41,105 - INFO - Leakage : f(['N', 'T', 'Y'])
2025-07-26 16:31:41,106 - INFO - fitting model for propensity: 'propensity'
2025-07-26 16:31:41,282 - INFO - fitting model for propensity: 'propensity_leakage'
2025-07-26 16:31:41,492 - INFO - fitting model for imputation
2025-07-26 16:31:41,604 - INFO - fitting model for imputation


,Naive,ATE_Propensity,ATE_Impute,ATE_Propensity_Leakage,ATE_Impute_Leakage
0,0.69138,0.500198,0.500184,0.499534,0.432704


## Bidirectional Causality

Movie A can appear before or after Movie B

In [17]:

SCM = {
    "R1" : lambda N: torch.rand(N),
    "R2" : lambda N: torch.rand(N),
    "U" :  lambda N: (torch.rand(N) < 0.5)*1.0,
    "A1":  lambda N, U, R1: (R1 < (0.1 + 0.4*U))*1.0,
    "B" :  lambda N, U, A1, R2: (R2 < (0.1 + 0.4*U + 0.4*A1))*1.0,
    "A2":  lambda N, U, A1, B: torch.where(A1>0.5, torch.zeros(N), (torch.rand(N) < (0.1 + 0.4*U + 0.4*B))*1.0),
    "A" :  lambda A1, A2: (A1+A2 > 0.5) * 1.0
}

values = eval_scm(SCM, 100000)

ground_truth = ground_truth_ate(SCM, values, "A1","B")

enrich_propensity(values, "A1", ["U"])
enrich_propensity(values, "A", ["U"], "propensity_alt")


show(GroundTruth=ground_truth, 
     Naive=CondMeanDiff("A1","B")(values),
     ATE_Stratified=ATEStratified("A1","B", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("A1","B", "propensity")(values),
     ATE_Impute=ATEImpute("A1","B",["U"])(values),
     ATE_Propensity_Alt=ATEPropensity("A1","B", "propensity_alt")(values),
     ATE_Propensity_Filtered=ATEPropensity("A","B", "propensity")(apply_filter(values, (values["A2"] < 0.5))),
     ATE_Impute_TisA=ATEImpute("A","B",["U"])(values),
    )



2025-07-26 16:31:41,763 - INFO - R1 : f(['N'])
2025-07-26 16:31:41,764 - INFO - R2 : f(['N'])
2025-07-26 16:31:41,766 - INFO - U : f(['N'])
2025-07-26 16:31:41,768 - INFO - A1 : f(['N', 'U', 'R1'])
2025-07-26 16:31:41,769 - INFO - B : f(['N', 'U', 'A1', 'R2'])
2025-07-26 16:31:41,770 - INFO - A2 : f(['N', 'U', 'A1', 'B'])
2025-07-26 16:31:41,772 - INFO - A : f(['A1', 'A2'])
2025-07-26 16:31:41,773 - INFO - Calculating ground truth
2025-07-26 16:31:41,773 - INFO - DO A1
2025-07-26 16:31:41,774 - INFO - A1 : f(['N'])
2025-07-26 16:31:41,774 - INFO - B : f(['N', 'U', 'A1', 'R2'])
2025-07-26 16:31:41,774 - INFO - A2 : f(['N', 'U', 'A1', 'B'])
2025-07-26 16:31:41,774 - INFO - A : f(['A1', 'A2'])
2025-07-26 16:31:41,774 - INFO - DO A1
2025-07-26 16:31:41,774 - INFO - A1 : f(['N'])
2025-07-26 16:31:41,781 - INFO - B : f(['N', 'U', 'A1', 'R2'])
2025-07-26 16:31:41,781 - INFO - A2 : f(['N', 'U', 'A1', 'B'])
2025-07-26 16:31:41,781 - INFO - A : f(['A1', 'A2'])
2025-07-26 16:31:41,781 - INFO - fi

,GroundTruth,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute,ATE_Propensity_Alt,ATE_Propensity_Filtered,ATE_Impute_TisA
0,0.4009,0.59334,0.402746,0.402812,0.402742,0.336022,0.616972,0.496424
